In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import datetime
import os
import json

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

from sklearn.metrics import mean_squared_error

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Prepare Data

In [2]:
rawPrice=pd.read_csv('./data/price.csv')
priceTitle = [d[0:4]+'-'+d[4:6]+'-'+d[6:] for d in rawPrice.columns[1:]]
priceTitle.insert(0, 'Date')
rawPrice.columns = priceTitle
rawPrice.set_index(['Date'], inplace=True)
rawPrice = rawPrice.T
rawPrice.to_csv('./processed_data/processed_price.csv')

rawPrice.index = pd.to_datetime(rawPrice.index, format="%Y-%m-%d")
print(rawPrice.shape)
rawPrice.head()

(1905, 3192)


Date        2       4       5       6       7       8       9       10      \
2014-01-02    7.99   11.81    2.49    4.87   14.04    9.00    9.40    7.70   
2014-01-03    7.84   11.80    2.46    4.73   14.00    8.95    9.15    7.76   
2014-01-06    7.48   12.30    2.38    4.46   14.04    8.51    9.07    7.22   
2014-01-07    7.43   12.23    2.45    4.45   14.58    8.39    9.05    7.24   
2014-01-08    7.42   12.23    2.41    4.39   14.62    8.23    8.89    7.38   

Date        11      12      14      16      17      19      20      21      \
2014-01-02    7.82    8.15   10.30    3.84    4.62    7.75    6.62    5.36   
2014-01-03    7.64    8.02   10.13    3.77    4.69    7.72    6.59    5.32   
2014-01-06    7.36    7.57    9.68    3.67    4.92    7.49    6.30    5.36   
2014-01-07    7.37    7.53    9.73    3.68    5.17    7.60    6.35    5.61   
2014-01-08    7.21    7.51    9.66    3.87    5.43    7.48    6.35    5.71   

Date        23      25      26      27      28      30      31      32      \
2014-01-02    7.90    8.80    7.53    5.47   46.14    6.14    3.68    8.86   
2014-01-03    7.98    8.82    7.53    5.34   45.33    6.01    3.62    8.76   
2014-01-06    8.04    8.40    7.37    5.22   48.10    5.74    3.53    8.47   
2014-01-07    7.89    8.25    7.35    5.20   48.20    5.74    3.54    8.24   
2014-01-08    8.05    8.19    7.42    5.26   48.20    5.72    3.52    8.42   

Date        34      35      36      37      38      39      40      42      \
2014-01-02    4.21   10.59    2.88    4.11   12.58   15.48    4.17   25.60   
2014-01-03    4.23   10.58    2.84    4.22   12.50   15.72    4.10   25.26   
2014-01-06    4.09   10.32    2.78    4.17   12.18   15.20    4.05   24.89   
2014-01-07    4.12   10.35    2.79    4.18   11.93   15.66    4.05   24.99   
2014-01-08    3.97   10.24    2.77    4.12   12.75   15.59    4.03   24.98   

Date        45      46      48      49      50      55      56      58      \
2014-01-02    8.00    4.47   11.13   72.84   11.56    5.72   19.39    4.96   
2014-01-03    7.92    4.39   11.15   71.40   11.57    5.61   19.48    5.14   
2014-01-06    7.53    4.32   10.80   70.36   11.12    5.40   18.94    4.79   
2014-01-07    7.46    4.25   10.91   77.40   11.80    5.47   18.37    4.71   
2014-01-08    7.44    4.31   11.83   76.78   12.13    5.46   18.30    4.77   

Date        59      60      61      62      63      65      66      68      \
2014-01-02    4.98     NaN    8.31   11.89   13.80   13.41    3.90    5.42   
2014-01-03    4.80     NaN    8.27   12.35   14.07   13.50    3.92    5.44   
2014-01-06    4.57     NaN    7.94   11.93   13.63   13.08    3.84    5.21   
2014-01-07    4.49     NaN    8.01   11.89   13.37   13.21    3.95    5.18   
2014-01-08    4.47     NaN    8.00   12.42   13.41   13.29    3.95    5.09   

Date        69      70      78      88      89      90      96      99      \
2014-01-02    5.26   10.25    7.53    6.38    4.34    6.76    5.39    8.60   
2014-01-03    5.15   10.16    7.60    6.19    4.27    6.69    5.28    8.47   
2014-01-06    4.98    9.80    7.42    5.96    4.29    6.34    5.01    8.16   
2014-01-07    4.96    9.82    7.50    6.04    4.28    6.42    5.18    8.19   
2014-01-08    4.94   10.12    7.53    5.92    4.29    6.40    5.26    8.13   

Date        100     150     151     153     155     156     157     158     \
2014-01-02    2.32    5.50    7.33     NaN    3.23   21.48    5.39    4.19   
2014-01-03    2.29    5.45    7.34     NaN    3.23   21.50    5.33    4.14   
2014-01-06    2.27    5.18    7.12     NaN    3.18   20.75    5.16    3.97   
2014-01-07    2.27    5.10    7.01     NaN    3.13   20.64    5.18    4.00   
2014-01-08    2.28    5.06    6.93     NaN    3.13   21.44    5.17    3.93   

Date        159     301     333     338     400     401     402     403     \
2014-01-02    7.21    3.25   48.92   18.74   31.37    8.59    5.16   15.82   
2014-01-03    6.98    3.23   48.07   18.33   31.65    8.42    5.06   15.70   
2014-01-06    

In [3]:
cluster = pd.read_csv('./processed_data/cluster.csv')
cluster.columns = ['ts_code', 'cluster_id']
cluster = cluster.sort_values(by='cluster_id')
cluster.to_csv('./processed_data/processed_cluster.csv')

print(cluster.shape)
cluster.head()

(3191, 2)


,ts_code,cluster_id
1228,2815,0
415,968,0
1470,300082,0
2506,600330,0
1333,2930,0


In [4]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Train

In [5]:
def train(m, index, epo, verbose=False):
    stock = rawPrice[[index]]
    stock = stock.dropna()
    if verbose:
        sns.set()
        plt.figure(figsize=(20,10))
        plt.plot(stock, color = "black")
        plt.ylabel("Price")
        plt.xlabel("Date")
        plt.xticks(rotation=45)
        plt.show()
    
    X, y =split_sequence(stock.T.values[0], 20)
    X = X.reshape((X.shape[0], X.shape[1], n_features))
    X_train = X[0:int(len(X)*0.8)]
    X_test = X[int(len(X)*0.8):]
    y_train = y[0:int(len(X)*0.8)]
    y_test = y[int(len(X)*0.8):]
    print('Train sample Num: ', len(X_train),', Test sample Num: ', len(X_test))
    
    m.fit(X_train, y_train, epochs=epo, verbose=0)
    
    res = [m.predict(X_test[i].reshape((1, n_steps, n_features)), verbose=0)[0][0] for i in range(len(X_test))]
    mse_ = mean_squared_error(res, y_test)
    mse_dict[idx][1] = mse_
    print('MSE for stock{}: '.format(idx), mean_squared_error(res, y_test))

In [6]:
def train_cluster(m, clusterIndex, epo, verbose=False):
    X_all = []
    y_all = []
    X_test_split = {}
    y_test_split = {}
    for idx in cluster[cluster['cluster_id'] == clusterIndex]['ts_code'].values:
        stock = rawPrice[[idx]]
        stock = stock.dropna()
        X, y =split_sequence(stock.T.values[0], 20)
        X = X.reshape((X.shape[0], X.shape[1], n_features))
        X_test_split[idx] = X[int(len(X)*0.8):]
        y_test_split[idx] = y[int(len(X)*0.8):]
        
        for i in range(X.shape[0]):
            X_all.append(X[i])
            y_all.append(y[i])
        
    X_train = X_all[0:int(len(X_all)*0.8)]
    X_test = X_all[int(len(X_all)*0.8):]
    y_train = y_all[0:int(len(y_all)*0.8)]
    y_test = y_all[int(len(y_all)*0.8):]
    print('Train sample Num: ', len(X_train),', Test sample Num: ', len(X_test))
    
    m.fit(np.array(X_train), np.array(y_train), epochs=epo, verbose=verbose)
    
    for idx in X_test_split.keys():
        res = [m.predict(X_test_split[idx][i].reshape((1, n_steps, n_features)), verbose=0)[0][0] for i in range(len(X_test_split[idx]))]
        mse_ = mean_squared_error(res, y_test_split[idx])
        mse_dict[idx] = [mse_, 0]
        print('MSE for stock{}: '.format(idx), mean_squared_error(res, y_test_split[idx]))

# Main

In [7]:
n_steps = 20
n_features = 1
mse_dict = {}

In [8]:
# Our method
for clu_id in list(set(cluster['cluster_id'].values)):
    model = Sequential()
    model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    
    print('\nStocks in cluster{}: '.format(clu_id), cluster[cluster['cluster_id'] == clu_id]['ts_code'].values)
    train_cluster(model, clu_id, 20, verbose=False)


Stocks in cluster0:  [  2815    968 300082 600330   2930 600195 600995]
Train sample Num:  8780 , Test sample Num:  2196
MSE for stock2815:  0.13513301799742108
MSE for stock968:  0.0880315520092321
MSE for stock300082:  0.5073040785595854
MSE for stock600330:  0.10896786553736362
MSE for stock2930:  0.3200799610952653
MSE for stock600195:  0.21850049858586562
MSE for stock600995:  0.2112039974070223

Stocks in cluster1:  [   553 600079 600528   2459 600699]
Train sample Num:  7066 , Test sample Num:  1767
MSE for stock553:  0.08750816208870561
MSE for stock600079:  1.6948915566273333
MSE for stock600528:  0.07489602060264923
MSE for stock2459:  4.739132696597997
MSE for stock600699:  1.1634729231136391

Stocks in cluster2:  [  2115   2400 300242]
Train sample Num:  4035 , Test sample Num:  1009
MSE for stock2115:  0.04211858154499471
MSE for stock2400:  0.10554764498404778
MSE for stock300242:  0.06013535303829737

Stocks in cluster3:  [300422 300284 300166    948 300040   2895 30015

In [9]:
# common_models = list()
# for idx in cluster[cluster['cluster_id'] == 0]['ts_code'].values:
#     model = Sequential()
#     model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
#     model.add(LSTM(50, activation='relu'))
#     model.add(Dense(1))
#     model.compile(optimizer='adam', loss='mse')
    
#     print('='*20+'Stock '+str(idx)+'='*20)
#     train(model, idx, 20)
#     common_models.append(model)

In [11]:
# Baseline
for idx in rawPrice.columns.to_list():
    if idx == 563:
        continue
    model = Sequential()
    model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    
    train(model, idx, 20)


Train sample Num:  1400 , Test sample Num:  350
MSE for stock2:  0.34087019914905137
Train sample Num:  1340 , Test sample Num:  336
MSE for stock4:  0.761845989604276
Train sample Num:  1427 , Test sample Num:  357
MSE for stock5:  0.014361400220410618
Train sample Num:  1414 , Test sample Num:  354
MSE for stock6:  0.06935402598290578
Train sample Num:  1055 , Test sample Num:  264
MSE for stock7:  0.2797663693063282
Train sample Num:  1323 , Test sample Num:  331
MSE for stock8:  0.053277671988443166
Train sample Num:  1435 , Test sample Num:  359
MSE for stock9:  0.6317965870361147
Train sample Num:  1424 , Test sample Num:  356
MSE for stock10:  0.0234481172496128
Train sample Num:  1508 , Test sample Num:  377
MSE for stock11:  1.3775567727003515
Train sample Num:  1488 , Test sample Num:  372
MSE for stock12:  0.11552532193952848
Train sample Num:  1508 , Test sample Num:  377
MSE for stock14:  0.2589121150726701
Train sample Num:  1432 , Test sample Num:  358
MSE for stock16:  

In [12]:
for idx in mse_dict.keys():
    mse_dict[idx].append(mse_dict[idx][0] - mse_dict[idx][1])
    print('MSE Improvement for stock {} is {}'.format(idx, mse_dict[idx][2]))

MSE Improvement for stock 2815 is -0.018946747996024582
MSE Improvement for stock 968 is -0.022987637536003108
MSE Improvement for stock 300082 is -0.61553077951982
MSE Improvement for stock 600330 is -0.0643625001237548
MSE Improvement for stock 2930 is 0.05805565144703839
MSE Improvement for stock 600195 is 0.056674649623621426
MSE Improvement for stock 600995 is 0.03967400871852378
MSE Improvement for stock 553 is 0.0468548556680694
MSE Improvement for stock 600079 is 0.6581704069675047
MSE Improvement for stock 600528 is -0.2805399582562497
MSE Improvement for stock 2459 is -121.79901324150607
MSE Improvement for stock 600699 is 0.41172746341498045
MSE Improvement for stock 2115 is -0.05341860875781152
MSE Improvement for stock 2400 is -0.02890669084451511
MSE Improvement for stock 300242 is -0.02275504454276188
MSE Improvement for stock 300422 is -0.9764838339439339
MSE Improvement for stock 300284 is -0.6219458923283744
MSE Improvement for stock 300166 is -0.005479894713650013
MS

In [14]:
res = {}
for k in mse_dict.keys():
    res[str(k)] = mse_dict[k]
    
jsObj = json.dumps(res)  
fileObject = open('./processed_data/res.json', 'w')  
fileObject.write(jsObj)  
fileObject.close()  